# 5- Modeling using Sentiment Prediction and Recommender:
#### Amin Khoeini

---

In the final step of this project, we combine the Sentiment prediction model with the hybrid movie recommendation.

The final goal of this project was to recieve a review from a user, and find out how much he love the movie.
Then if we predict that the user love the movie, we suggest a list of movie to the user to watch. That will be possible
by combining our models.

Therefore our imput will be:
- User ID
- Movie Title
- Review

We used this input, and feeds it to the function that we create to combine our two models. This function first predict the sentiment of the review:
- If the review wasn't positive our input will states that this user didn't enjoy this movie.
- If the review was positive then we enter the next level:
 * We feed the user ID and movie title to the hybrid recommder
 * The recommnder will provide the top movies suggestion for this user.
 
 
So in case of the user enjoied the movie, our output will be the movie suggestion list.

In [1]:
# Import the recommender
import Hybrid_Recommender

Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [2]:
# Import the libraries
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [3]:
# create a function for cleaning the new reviews
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()
import string as st
def clean(text):
    text = text.lower()
    t1 = [w for w in text if w not in st.punctuation]
    t1 = ''.join(t1)
    stopword= nltk.corpus.stopwords.words('english')
    stopword.remove('not')
    t2 =' '.join(w for w in nltk.word_tokenize(t1) if w not in stopword)
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(t2)])
    

In [4]:
# Import the Tfidf vectorizer and Sentiment Prediction model
model = pickle.load(open('sentiment_model.sav','rb'))
tfidf = pickle.load(open('tfidf.sav','rb'))

In [23]:
# Create a function to apply both sentiment prediciton and recommender to a new review
def sentiment_recommender(user_id,movie_title,review):
    ''' This function will combine the two model we create before. The input is the User_id,Movie title and
    The review that user write for the movie.
        
        First step, we use the sentiment prediction model to see if the user enjoy the movie or not.
        If not the function just print that user didn't enjoy this movie.
        If the model predict the user enjoy the movie, we go to the next level.
        In next level, we use the user ID and movie title and our hybrid movie recommendition engine
        to produce a top movie suggestion for the user and print it as the final output.
    '''
    
    review = clean(review)
    vector = tfidf.transform([review])
    s =  model.predict(vector)
    if s == 'NOT LOVE IT':
        print ("The user did not enjoy",movie_title,"therefor the system has no recomandation based on that review.")
        return
    else:
        print("Based on the review, This user loved the",movie_title,"therefor the system recommand the below list for the user to watch: \n\n",
             Hybrid_Recommender.recommender(user_id,movie_title).tolist())
        return

### Using the final model,create some example, and evalute the performance:
___

First we use two review that the sentement is strong. First positive then negetive and use a same user_id and see how the model works.

In [24]:
user_id= 20
movie_title= 'Kill Bill: Vol. 2 (2004)'
review = "After killing O-Ren Ishii (Lucy Liu) and Vernita Green (Vivica A. Fox), \
          The Bride chases Budd (Michael Madsen) and Elle (Daryl Hanna). Then, \
          she finds Bill (David Carradine), where a surprise waits for her. \
          The second part of Kill Bill is a disappointing and overrated sequel. \
          The reason and the massacre in the Texas church are finally presented and is the best part \
          of this volume 2. The first part (Vol. 1) of this story was full of action and very funny, \
          but there is too much comic book cheap philosophy talk and a corny conclusion in this second part. \
          In Brazil, `Kill Bill Vol. 2' has not been released in the movie theaters yet. \
          I saw it in an imported DVD and I really did not like it, since I expected much more from \
          Quentin Tarantino. My vote is six."

sentiment_recommender(user_id,movie_title,review)

Based on the review, This user loved the Kill Bill: Vol. 2 (2004) therefor the system recommand the below list for the user to watch: 

 ['Jackie Brown (1997)', 'Pulp Fiction (1994)', 'Sin City (2005)', 'Inglourious Basterds (2009)', 'Kill Bill: Vol. 1 (2003)', 'Reservoir Dogs (1992)', 'The Hateful Eight (2015)', 'Django Unchained (2012)', 'The China Syndrome (1979)', 'From Dusk Till Dawn (1996)']


In [25]:
user_id= 20
movie_title= 'Kill Bill: Vol. 2 (2004)'
review = "OK, I don't really want Quentin Tarantino to be killed - actually, I am a big fan of his earlier films. \
But the 'Kill Bill' films are not in their league, all style and no substance. Volume 1 at least had a fast pace, \
its innovations coming quick and fast; but Volume 2 of this over-long project seems to add nothing new except for\
a layer of emotional interpretation that its grotesque, cartoonish characters can scarcely sustain. \
Volume 1 pastiched a huge range of different styles of film; Volume 2, at times, appears simply as a pastiche \
of a bad movie (and very hard to distinguish from a genuinely bad film to boot). Yet I can't help but feel that \
within these two movies (originally conceived as one very long film) there's a highly original shorter film that \
should have been made. When the saga ends with a credits sequence in which Tarantino appears to pay homage to \
himself, one can't help but wonder whether even the most talented of individuals don't sometimes benefit when \
someone else has the final cut."


sentiment_recommender(user_id,movie_title,review)

The user did not enjoy Kill Bill: Vol. 2 (2004) therefor the system has no recomandation based on that review.


The model working fine for the obvious review, and in the case a positive review recommands a good list, mostly from same director and other action movie for the user.

What if the user ID will change? 

In [26]:
user_id= 40
movie_title= 'Kill Bill: Vol. 2 (2004)'
review = "After killing O-Ren Ishii (Lucy Liu) and Vernita Green (Vivica A. Fox), \
          The Bride chases Budd (Michael Madsen) and Elle (Daryl Hanna). Then, \
          she finds Bill (David Carradine), where a surprise waits for her. \
          The second part of Kill Bill is a disappointing and overrated sequel. \
          The reason and the massacre in the Texas church are finally presented and is the best part \
          of this volume 2. The first part (Vol. 1) of this story was full of action and very funny, \
          but there is too much comic book cheap philosophy talk and a corny conclusion in this second part. \
          In Brazil, `Kill Bill Vol. 2' has not been released in the movie theaters yet. \
          I saw it in an imported DVD and I really did not like it, since I expected much more from \
          Quentin Tarantino. My vote is six."

sentiment_recommender(user_id,movie_title,review)

Based on the review, This user loved the Kill Bill: Vol. 2 (2004) therefor the system recommand the below list for the user to watch: 

 ['Hamburger Hill (1987)', 'The China Syndrome (1979)', 'Pulp Fiction (1994)', 'Reservoir Dogs (1992)', 'Sin City (2005)', 'Jackie Brown (1997)', 'Django Unchained (2012)', 'Inglourious Basterds (2009)', 'Grindhouse (2007)', 'The Hateful Eight (2015)']


Interestingly, the model suggest another two movies the top choice for this user, following the other films of the director. So the collaborative filter works fine too to include the taste of the user in the suggestion too.

---

Now let's get the recommendation for a classic movie that does not have lots of review in the database and see how the recommandation works.


In [27]:
# Use a real review with 8/10 score. And there is not a literall words in review (like Bad, Worse,Good,Best)
# still model predict it right and recommend a good list for it.

user_id= 450
movie_title= 'The Cabinet of Dr. Caligari (1920)'
review = "With a story as twisted, buckled and demented as the scenery and set design, and a tortured score \
to boot, was the lunatic in charge of the asylum? Must have been a moment to remember coming across this \
cinematic marvel back in 1920. I wonder how many of today's pieces of cinema will stand the test of 100 years,\
albeit with a lot more competition."

sentiment_recommender(user_id,movie_title,review)

Based on the review, This user loved the The Cabinet of Dr. Caligari (1920) therefor the system recommand the below list for the user to watch: 

 ['Enter the Dragon (1973)', 'What Ever Happened to Baby Jane? (1962)', 'The Natural (1984)', 'Murder by Death (1976)', 'Short Cuts (1993)', 'The Curse of the Cat People (1944)', 'Lions for Lambs (2007)', 'The VVitch: A New-England Folktale (2015)', 'The Spiral Staircase (1946)', 'Psycho II (1983)']


In [9]:
# Another review with 6/10 score. and model correctly lable it as NOT LOVE IT.
user_id= 4566
movie_title= 'The Cabinet of Dr. Caligari (1920)'
review = "The Cabinet of Dr Caligari is upheld as one of cinema's greats, and in a number of regards,\
I can understand why. It is hauntingly beautiful and wonderfully stylised. It's Expressionistic cinematography \
is wonderful and suspenseful; it's no wonder that this film has influenced many subsequent directors and \
producers many years later.However, whilst it surely wouldn't have seemed like this back in 1920, it is rather \
boring through contemporary eyes. I'm sorry if this makes me sound ignorant, but today it plays rather \
slow-paced and un-scary, even compared to some of the other greats of Weimar cinema (such as Nosferatu)\
.Not a bad film by any means (I would recommend it), but Caligari seems more something of historical value \
and academic study to me."


sentiment_recommender(user_id,movie_title,review)

The user did not enjoy The Cabinet of Dr. Caligari (1920) therefor the system has no recomandation based on that review.


Here for the positive review, we get the recommandation with the similar genre and content of the movie. There is only one classic film is in the recommadation list, probably beacase there are not many of the classic movie in the database and from the classic directors. Still the recommendation list is interesting and close to the target movie.

---

Now try a movie that is a part of a series and see the recommendation for it.

In [28]:
# 8/10 review.
user_id= 112
movie_title= 'Rocky IV (1985)'
review = "The peaks of the Rocky franchise are the first movie and the fourth movie. \
Rocky is the Academy Award-winning drama and Rocky IV is the exhilarating, montage-driven 80s popcorn flick. \
Yes, this is a silly movie, but Ivan Drago is an intimidating opponent, and I think we all shed a tear or two \
when Apollo died. If nothing else, this movie gave us one of the best workout soundtracks ever - thanks, Rocky IV!"


sentiment_recommender(user_id,movie_title,review)

Based on the review, This user loved the Rocky IV (1985) therefor the system recommand the below list for the user to watch: 

 ['Rocky (1976)', 'Antz (1998)', 'Rocky II (1979)', 'Rocky Balboa (2006)', 'First Blood (1982)', 'Nighthawks (1981)', 'Cop Land (1997)', 'Death Race 2000 (1975)', 'Rocky III (1982)', 'Demolition Man (1993)']


Get the most of the film on the series, plus other movies from the main actor.

---

Next we try to a less known movie from a super hero series, which got lots of negetive review from the user and see how the model perform on that.

In [29]:
#5/10 review.
user_id= 9877
movie_title= 'Superman IV: The Quest for Peace (1987)'
review = "Superman IV is not nearly as bad as the reviews suggest. The actors try really hard, \
particularly Christopher Reeve, Gene Hackman, and Margot Kidder, to make it work. The movie is watchable \
and the musical score is good. The movie is an improvement over the disappointing Superman III. \
However, Superman IV has major problems. The movie has obviously been cut from its original length \
make it incoherent at times. The special effects are below the standards set in the first two movies \
(even the third movie had decent effects). Maybe if the movie were restored to its original length, \
it would be better. I can only give this movie a 5/10. I wished it were better and hope someday they \
do restore this movie to its original length."


sentiment_recommender(user_id,movie_title,review)

The user did not enjoy Superman IV: The Quest for Peace (1987) therefor the system has no recomandation based on that review.


In [30]:
# 8/10 review. the review brings up all the bad things about the movie but still say he loved it 
# because it is superman movie. Model can't detect that.
user_id= 458
movie_title= 'Superman IV: The Quest for Peace (1987)'
review = "I cannot stand public opinion about the fourth and final Christopher Reeve Superman outing. \
Superman IV: The Quest for Peace, while not the best of the entire series, is better than Superman III \
and better than Richard Lester's cartoonish Superman II (in which Lois Lane actually says, I'm going to \
change into something more comfortable).First of all, people have claimed that IV is the most physically \
ridiculous of all four. This is not true. Yes, for a human to fly in space, they would have to survive being\
burned up leaving Earth's atmosphere, then survive the vacuum of space (in other words, two people would have \
been dead by the end of the movie). However, let's revisit the 1978 original, in which the Man of Steel \
flies around the world, making it spin backwards, which also makes time reverse. If that really happened, \
every living thing and object on the planet's face would be flung off into the cold depths of space.\
I won't linger on this anymore. Yes, The Quest for Peace has unbearable dialogue (in places), \
but it's classic Superman-- the cheese, the cheap effects, and the general cartoon feel. \
Plus, it does have Jon Cryer, in what is oddly his best performance of all time (you'd better believe it), \
and a solid message people still need to think about. In other words, come to believe a man can fly, \
stay for the inherent message of the movie: we would have to be very stupid to nuke our own planet."

sentiment_recommender(user_id,movie_title,review)

The user did not enjoy Superman IV: The Quest for Peace (1987) therefor the system has no recomandation based on that review.


In [31]:
#7/10 review.
user_id= 5675
movie_title= 'Superman IV: The Quest for Peace (1987)'
review = "This is the fourth and final Superman film with Christopher Reeve taking on the role, \
where he tries to stop the spread of nuclear weapons and battles Lex Luther (Gene Hackman) and his \
super-powered sidekick, Nuclear Man (Mark Pillow).Though the level of excitement and intrigue of this \
film doesn't match the first two, it is almost on par with the third and is still fun and fast-paced \
with neat special effects that showcase Superman's powers and great action scenes, something that sorely \
lacks in Superman Returns.The nuclear weapons plot I thought was clever and unique to a Superman movie. \
Total disarmament of nuclear weapons may not be feasible in the real world, but Superman's quest for \
peace among all countries is well-intended in the film. Reeve is great as Superman and Gene Hackman gave \
us another thrilling performance as super villain Lex Luther. In addition, the fight scenes between \
Nuclear Man and Superman were awesome - Nuclear Man looked like a force to be reckon with as he possessed \
the same level of powers and same physique as Superman.Overall, I think this film is a fitting ending to \
the Christopher Reeve saga."

sentiment_recommender(user_id,movie_title,review)

Based on the review, This user loved the Superman IV: The Quest for Peace (1987) therefor the system recommand the below list for the user to watch: 

 ['Raiders of the Lost Ark (1981)', '2001: A Space Odyssey (1968)', 'Predator (1987)', 'Iron Man (2008)', 'Spider-Man (2002)', 'Captain America: Civil War (2016)', 'Dead Man (1995)', 'The Ipcress File (1965)', 'Star Trek VI: The Undiscovered Country (1991)', 'Hot Shots! (1991)']


In [34]:
# Same Review but changing the user_id to see a diffrence in the recommendation.
user_id= 3434
movie_title= 'Superman IV: The Quest for Peace (1987)'
review = "This is the fourth and final Superman film with Christopher Reeve taking on the role, \
where he tries to stop the spread of nuclear weapons and battles Lex Luther (Gene Hackman) and his \
super-powered sidekick, Nuclear Man (Mark Pillow).Though the level of excitement and intrigue of this \
film doesn't match the first two, it is almost on par with the third and is still fun and fast-paced with \
neat special effects that showcase Superman's powers and great action scenes, something that sorely lacks\
in Superman Returns.The nuclear weapons plot I thought was clever and unique to a Superman movie. \
Total disarmament of nuclear weapons may not be feasible in the real world, but Superman's quest for \
peace among all countries is well-intended in the film. Reeve is great as Superman and Gene Hackman \
gave us another thrilling performance as super villain Lex Luther. In addition, the fight scenes between \
Nuclear Man and Superman were awesome - Nuclear Man looked like a force to be reckon with as he possessed \
the same level of powers and same physique as Superman.Overall, I think this film is a fitting ending to the \
Christopher Reeve saga."

sentiment_recommender(user_id,movie_title,review)

Based on the review, This user loved the Superman IV: The Quest for Peace (1987) therefor the system recommand the below list for the user to watch: 

 ['2001: A Space Odyssey (1968)', 'Raiders of the Lost Ark (1981)', 'Captain America: Civil War (2016)', 'Predator (1987)', 'Iron Man (2008)', 'Spider-Man (2002)', 'Star Trek VI: The Undiscovered Country (1991)', 'The Ipcress File (1965)', 'Dead Man (1995)', 'Rumble in the Bronx (1995)']


The only time the model predict the wrong sentiment for the review was for this film.
The secound reviewer brough all the negetive feed-back for this movie but at the end he said he loves this film because it is another super-man movie. Because there are lots of negetive words in the review model predict that user did not like it.

For the positive review, we got the interesting suggestion. Another super hero movie mostly from recent time, combine with some similar genre movie. And it is slightly diffrent for the diffrent user.